## Repressilator
Example to test handling rational functions (such as hill functions) in Oscar.

In [160]:
using CRNAnalysis
using Catalyst
using Oscar
import Symbolics

In [116]:
function repressilator(n::Int; name=:Repressilator)
    t = default_t()
    @reaction_network $name begin
        @species X(t) Y(t) Z(t)
        hillr(Z, β, 1, $n), 0 --> X
        hillr(X, β, 1, $n), 0 --> Y
        hillr(Y, β, 1, $n), 0 --> Z
        1, X --> 0
        1, Y --> 0
        1, Z --> 0
    end
end
rn = repressilator(2)
nlsys = convert(NonlinearSystem, rn)

Model Repressilator:
Equations (3):
  3 standard: see equations(Repressilator)
Unknowns (3): see unknowns(Repressilator)
  X(t)
  Y(t)
  Z(t)
Parameters (1): see parameters(Repressilator)
  β

Steps:
- check if mass-action/polynomial
  - if all reactions are, continue as normal
- if not, check for rational functions
  - convert to polynomials
    - factor out denominator of rational functions (LHS = 0)
- otherwise, error

In [165]:
vars = Catalyst.unknowns(nlsys)
params = parameters(nlsys)
eqns = equations(nlsys)
polyn_rhs = expand_rational_equation.(eqns)

3-element Vector{Num}:
 -X(t) + β - X(t)*(Z(t)^2)
 -Y(t) + β - Y(t)*(X(t)^2)
 -Z(t) + β - (Y(t)^2)*Z(t)

In [166]:
# Oscar Ideal
I = polynomial_ideal(polyn_rhs, vars, params)

Ideal generated by
  -X*Z^2 - X + β
  -X^2*Y - Y + β
  -Y^2*Z - Z + β